In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
race_results_df = spark.read.format("delta").load(f"{incpresentation_folder_path}/race_results_dt") \
.filter(f"file_date = '{v_file_date}'") 

In [0]:
race_year_list = df_column_to_list(race_results_df, 'race_year')

In [0]:
from pyspark.sql.functions import col

race_results_df = spark.read.format("delta").load(f"{incpresentation_folder_path}/race_results_dt") \
.filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, when, count, col

driver_standings_df = race_results_df \
.groupBy("race_year", "driver_name", "driver_nationality") \
.agg(sum("points").alias("total_points"),
     count(when(col("position") == 1, True)).alias("wins"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = driver_standings_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
display(final_df)

race_year,driver_name,driver_nationality,total_points,wins,rank
2021,Lewis Hamilton,British,44.0,1,1
2021,Max Verstappen,Dutch,43.0,1,2
2021,Lando Norris,British,27.0,0,3
2021,Charles Leclerc,Monegasque,20.0,0,4
2021,Valtteri Bottas,Finnish,16.0,0,5
2021,Daniel Ricciardo,Australian,14.0,0,6
2021,Carlos Sainz,Spanish,14.0,0,6
2021,Sergio Pérez,Mexican,10.0,0,8
2021,Pierre Gasly,French,6.0,0,9
2021,Lance Stroll,Canadian,5.0,0,10


In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_year = src.race_year"
merge_delta_data(final_df, 'f1_inc_presentation', 'driver_standings_dt', incpresentation_folder_path, merge_condition, 'race_year')